Name: Gaurav B Raskar
Contact: 8262885459
Email: 117tree3@gmail.com

In [56]:
import os
import configparser

from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

from langchain_core.messages import HumanMessage, SystemMessage

config = configparser.ConfigParser()
config.read('../config.ini')
groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')

In [97]:
import movieScriptPicker
import asyncio
from langchain.prompts import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

parser = StrOutputParser()

system_template = "You are a movie story writer and the Actor in movie is {Actor}, Location is {location} and Theme is {theme}"
userInput = "Give a short 5 sentence story."
prompt_template = ChatPromptTemplate.from_messages([("system", system_template),("user",userInput)])
StoryOpt = {"Actor": movieScriptPicker.actor_picker(),"location": movieScriptPicker.location_picker(),"theme": movieScriptPicker.theme_picker()}

store1 = InMemoryChatMessageHistory()
store2 = InMemoryChatMessageHistory()

model1 = ChatCohere(model="command-r-plus")
story1 = (prompt_template |model1 | parser).invoke(StoryOpt)
store1.add_message(SystemMessage(story1))

model2 = ChatGroq(model="llama3-8b-8192")
story2 = (prompt_template |model2 | parser).invoke(StoryOpt)
store2.add_message(SystemMessage(story2))

print(StoryOpt)
print("*Cohere Story:\n\t",story1)
print("\n*Groq Story:\n\t",story2)

async def getMessage():
    await asyncio.sleep(2)
    a= await store1.aget_messages()
    b= await store2.aget_messages()
    return a,b
while True:
    print("\nGive a change -> ")
    userInput = input("Want any change in story? Press 'n' to Exit ")
    if userInput=='n': break
    else:
        #str = "Change this story as given."+ userInput
        store1.add_message(HumanMessage(content=userInput))
        store2.add_message(HumanMessage(content=userInput))
        msgs1, msgs2 = await getMessage()

        chat1 = (model2 | parser).invoke(msgs1)
        chat2 = (model2 | parser).invoke(msgs2)
        print("\n*Cohere Story:\n\t",chat1)
        print("\n*Groq Story:\n\t",chat2)
        store1.add_message(SystemMessage(chat1))
        store2.add_message(SystemMessage(chat2))
        

#Press "n" to exit the loop...

{'Actor': 'Shahrukh Khan', 'location': 'Tokyo', 'theme': 'Crime'}
*Cohere Story:
	 In the bustling city of Tokyo, a charismatic and notorious criminal, played by Shahrukh Khan, is on a daring crime spree. With each heist, he leaves a unique signature, a playful clue for the authorities. As the stakes rise, a determined detective enters the scene, vowing to bring Khan's character to justice. The cat-and-mouse game that ensues takes them through the vibrant and neon-lit streets of Tokyo, where secrets and surprises await at every turn. In a thrilling climax, Khan's character must decide between his life of crime and the possibility of redemption.

*Groq Story:
	 In the neon-lit streets of Tokyo, Shahrukh Khan's character, a suave and cunning con artist, teams up with a beautiful Japanese thief to pull off the heist of a lifetime. Their target is a priceless diamond necklace, hidden away in a heavily guarded safe. As they navigate the city's underground criminal network, they find themsel